In [1]:
!pip install python_dotenv
!pip install os
!pip install requests
!pip install gql
!pip install requests-toolbelt
!pip install re
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import os
from dotenv import load_dotenv
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import re
import pandas as pd

In [3]:
#carrega a chave da API
load_dotenv("../.env")
token = os.getenv('GH_TOKEN')
if token is None:
    raise ValueError("GitHub token is not set. Check your .env file.")

In [4]:
# le a query
with open('../queries/bitcoin-core.graphql', 'r') as file:
    query = file.read()
    
# inicializa cliente
transport = RequestsHTTPTransport(
    url='https://api.github.com/graphql',
    headers={'Authorization': f'token {token}'}
)
client = Client(transport=transport, fetch_schema_from_transport=True)

#executa query
query = gql(query)
result = client.execute(query)
result


{'repository': {'name': 'bitcoin',
  'description': 'Bitcoin Core integration/staging tree',
  'issues': {'edges': [{'node': {'title': 'JSON-RPC support for mobile devices ("ultra-lightweight" clients)',
      'body': 'The idea is to enable mobile or other not-always-connected-to-the-network clients, where the wallet is kept (encrypted, probably) on the mobile device, and the device periodically communicates with a bitcoin-network-connected node to:\n- Update balance for keys in its wallet\n- Submit signed transactions to the network\n',
      'createdAt': '2010-12-19T16:17:53Z',
      'comments': {'edges': [{'node': {'body': 'Yes, very important. If we enable mobile devices, pocket wallets will be soon obsolete ;-) GUI for smart-phones should include some QR reader, such as portapayment.com \n',
          'createdAt': '2011-02-07T13:43:11Z'}},
        {'node': {'body': '"Submit signed transactions to the network"\n\nI want that as well, for Google App Engine, where you cannot particip

In [5]:
#extrai issues e comentrios
issues = result.get('repository', {}).get('issues', {}).get('edges', [])
data = []

for issue_edge in issues:
    issue = issue_edge.get('node', {})
    data.append(issue.get('body', ''))
    
    comments = issue.get('comments', {}).get('edges', [])
    for comment_edge in comments:
        comment = comment_edge.get('node', {})
        data.append(comment.get('body', ''))

In [6]:
#função para contagem de palavras
def count_occurrences(text, word):
    return len(re.findall(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE))

# Combine all data into a single string
combined_text = ' '.join(data)

#conta a quantidade de ocorrencias de segurança
security_count = count_occurrences(combined_text, 'security')

print(f"The word 'security' occurs {security_count} times in the repository.")


The word 'security' occurs 3 times in the repository.


In [7]:
#função que combina os nodes em uma str
def fetch_all_issues():
    issue_cursor = None
    comment_cursor = None
    combined_text = []
    
    while True:
        variables = {
            'issueCursor': issue_cursor,
            'commentCursor': comment_cursor
        }

        #executa query
        try:
            result = client.execute(query, variable_values=variables)
        except Exception as e:
            print(f"An error occurred: {e}")
            break
        
        issues = result.get('repository', {}).get('issues', {}).get('edges', [])
        
        for issue_edge in issues:
            issue = issue_edge.get('node', {})
            combined_text.append(issue.get('body', ''))

            #append o comentario na string
            comments = issue.get('comments', {}).get('edges', [])
            for comment_edge in comments:
                comment = comment_edge.get('node', {})
                combined_text.append(comment.get('body', ''))
            
            #prox pag de commentarios
            page_info = issue.get('comments', {}).get('pageInfo', {})
            comment_cursor = page_info.get('endCursor', None)
            if not page_info.get('hasNextPage', False):
                comment_cursor = None
        
        #prox pagina de issues
        page_info = result.get('repository', {}).get('issues', {}).get('pageInfo', {})
        issue_cursor = page_info.get('endCursor', None)
        if not page_info.get('hasNextPage', False):
            break
    
    return ' '.join(combined_text)

combined_text = fetch_all_issues()

In [9]:
#função para contagem de palavras
def count_occurrences(text, word):
    return len(re.findall(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE))

#contagem da palavra segurança
security_count = count_occurrences(combined_text, 'security')

print(f"The word 'security' occurs {security_count} times in the repository.")



The word 'security' occurs 1454 times in the repository.


In [10]:
#função para contagem de palavras
def count_occurrences(text, words):
    word_counts = {word: len(re.findall(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE)) for word in words}
    return word_counts

# conta e define palavras a serem contadas
ethical_words = ['security', 'privacy', 'ethics', 'confidentiality', 'integrity']
word_counts = count_occurrences(combined_text, ethical_words)

#Imprime as ocorrencias de palavras
for word, count in word_counts.items():
    print(f"The word '{word}' occurs {count} times in the repository.")


The word 'security' occurs 1454 times in the repository.
The word 'privacy' occurs 467 times in the repository.
The word 'ethics' occurs 5 times in the repository.
The word 'confidentiality' occurs 4 times in the repository.
The word 'integrity' occurs 54 times in the repository.


In [11]:
def fetch_all_issues():
    issue_cursor = None
    comment_cursor = None
    issues = []
    comments = []
    
    while True:
        variables = {
            'issueCursor': issue_cursor,
            'commentCursor': comment_cursor
        }
        
        try:
            result = client.execute(query, variable_values=variables)
        except Exception as e:
            print(f"An error occurred: {e}")
            break
        
        issues_data = result.get('repository', {}).get('issues', {}).get('edges', [])
        
        for issue_edge in issues_data:
            issue = issue_edge.get('node', {})
            issues.append(issue.get('body', ''))
            
            comments_data = issue.get('comments', {}).get('edges', [])
            for comment_edge in comments_data:
                comment = comment_edge.get('node', {})
                comments.append(comment.get('body', ''))
            
            # Update the comment cursor for pagination
            page_info = issue.get('comments', {}).get('pageInfo', {})
            comment_cursor = page_info.get('endCursor', None)
            if not page_info.get('hasNextPage', False):
                comment_cursor = None
        
        # Update the issue cursor for pagination
        page_info = result.get('repository', {}).get('issues', {}).get('pageInfo', {})
        issue_cursor = page_info.get('endCursor', None)
        if not page_info.get('hasNextPage', False):
            break
    
    return issues, comments

issues, comments = fetch_all_issues()
total_issues = len(issues)
total_comments = len(comments)

In [12]:
# Cell 4: Define a function to count the number of issues and comments containing specific words
def count_contains(text_list, words):
    counts = {word: {'issues': 0, 'comments': 0} for word in words}
    
    for text in text_list:
        for word in words:
            if re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE):
                if text in issues:
                    counts[word]['issues'] += 1
                elif text in comments:
                    counts[word]['comments'] += 1

    return counts

# List of ethical-related words to search for
ethical_words = ['security', 'privacy', 'ethics', 'confidentiality', 'integrity']

# Count occurrences in issues and comments
word_counts = count_contains(issues + comments, ethical_words)

# Print the results
for word, counts in word_counts.items():
    print(f"The word '{word}' appears in {counts['issues']} issues and {counts['comments']} comments.")
    print(f"Percentage of issues containing '{word}': {counts['issues'] / total_issues * 100:.2f}%")
    print(f"Percentage of comments containing '{word}': {counts['comments'] / total_comments * 100:.2f}%")


The word 'security' appears in 633 issues and 304 comments.
Percentage of issues containing 'security': 7.82%
Percentage of comments containing 'security': 0.93%
The word 'privacy' appears in 71 issues and 152 comments.
Percentage of issues containing 'privacy': 0.88%
Percentage of comments containing 'privacy': 0.47%
The word 'ethics' appears in 1 issues and 4 comments.
Percentage of issues containing 'ethics': 0.01%
Percentage of comments containing 'ethics': 0.01%
The word 'confidentiality' appears in 2 issues and 2 comments.
Percentage of issues containing 'confidentiality': 0.02%
Percentage of comments containing 'confidentiality': 0.01%
The word 'integrity' appears in 24 issues and 23 comments.
Percentage of issues containing 'integrity': 0.30%
Percentage of comments containing 'integrity': 0.07%
